In [ ]:
from __future__ import print_function

import numpy as np
import glob


## Boilerplate path hack to give access to full clustered_SNe package
import sys, os
if __package__ is None:
    if os.pardir not in sys.path[0]:
        file_dir = os.getcwd()
        sys.path.insert(0, os.path.join(file_dir, 
                                        os.pardir, 
                                        os.pardir))
        
from clustered_SNe import data_dir_default
from clustered_SNe.analysis.constants import m_proton, pc, yr, M_solar, \
                                   metallicity_solar
from clustered_SNe.analysis.parse import Overview, RunSummary, \
                                         Inputs, \
                                         get_full_id_from_partial_id, \
                                         checkpoint_num_from_filename

from clustered_SNe.analysis.database_helpers import session, \
                                                Simulation, \
                                                Simulation_Inputs, \
                                                Simulation_Status


In [ ]:
data_dir = data_dir_default

In [ ]:
id_tuples = []
for filename in glob.glob(os.path.join(data_dir, "*_overview.dat")):
    overview = Overview(filename)
    if overview.num_SNe != 1:
        continue
    run_summary = RunSummary(overview.dirname, overview.id)
    current_status = session.query(Simulation_Status).get(overview.id).status
    delta_time = run_summary.times.max() - run_summary.times.min()
    if delta_time <=0 :
        raise RuntimeError("delta_time not positive")
    if np.any(run_summary.momentum < 0):
        id_tuples.append((overview.id, "negative momentum", current_status, delta_time))
        continue
    if run_summary.num_momenta_maxima() > 1:
        id_tuples.append((overview.id, "too many momentum maxima", current_status, delta_time))
        continue
    if run_summary.num_momenta_maxima == 0 :
        id_tuples.append((overview.id, "no momentum maxima?", current_status, delta_time))
        continue
    if not run_summary.is_time_resolved():
        id_tuples.append((overview.id, "not time resolved", current_status, delta_time))
        continue
        
    
    

In [ ]:
restart_dir = "restart_resolution"

qsub_systems = set(["SGE", "PBS"])
qsub_system = "PBS"
assert(qsub_system in qsub_systems)
    
f_submit = open("submit_script", mode="w")
f_submit.write("#!/bin/bash \n")

f_delete = open("delete_script", mode="w")
f_delete.write("#!/bin/bash \n")
f_delete.write("mkdir -p _tmp_ \n")

f_copy = open("copy_script", mode="w")
f_copy.write("#!/bin/bash \n")
f_copy.write("mkdir -p ../" + restart_dir + "\n")

for id, reason, status, delta_time in id_tuples:

    print("id: ", id)
        
    first_checkpoint_filename  = id + "_checkpoint_0000.dat"
    
    restart_checkpoints_needed = 99
    
    restart_time_needed = .3 * delta_time
    print("extra time needed: ", restart_time_needed, "[s]")

    print("")

    CFL = .025

    f_submit.write("qsub " +
                       "-v RESTART_ID=" + id + ","
                       + "RESTART_N_CHECKPOINTS=" + str(restart_checkpoints_needed) + ","
                       + "RESTART_DELTA_TIME=" + str(restart_time_needed) + ","
                       + "RESTART_CFL=" + str(CFL)
                    + " -N " + id + "_restart.batch "
                    + " ../scripts/" + qsub_system + "/restart.batch.env_variables \n")
    
    f_delete.write("cp -p       " + first_checkpoint_filename + " _tmp_/ \n")
    f_delete.write("rm          " + id + "_checkpoint_*.dat \n")
    f_delete.write("cp -p _tmp_/" + first_checkpoint_filename + " . \n")
    
    f_copy.write("cp -p " + first_checkpoint_filename + " ../" + restart_dir + " \n")
    f_copy.write("cp -p " + id + "_overview.dat       " + " ../" + restart_dir + " \n")
    f_copy.write("cp -p " + id + "_SNe.dat            " + " ../" + restart_dir + " \n")
    f_copy.write("cp -p " + id + "_inputs.dat         " + " ../" + restart_dir + " \n")
     
f_delete.write("rm -rf _tmp_ \n")
f_submit.close()
f_copy.close()
f_delete.close()

In [ ]:
for id in ids:
    session.query(Simulation_Status).filter(Simulation_Status.id == id).update({"status":"Running"})
    sim_status = session.query(Simulation_Status).get(id)
    print(sim_status.status)
    print(id)

In [ ]:
session.commit()